In [1]:
import numpy as np
import cv2
import glob
import os

In [2]:
chess_path = "chess"

In [3]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:6,0:7].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob(os.path.join(chess_path, '*.jpg'))
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (6,7), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        #increaqse accuracy of found corners
        corners2=cv2.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (6,7), corners2, ret)
        cv2.imshow('img', img)
        cv2.waitKey(0)
cv2.destroyAllWindows()

### Calibration

In [4]:
#return camera matrix, distortion parameters, rotation and translation vectors
ret, cameraMatrix, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Camera Calibrated: ", ret)
print("\nCamera Matrix:\n", cameraMatrix)
print("\nDistortion Parameters:\n", dist)
print("\nRotation Vectors:\n", rvecs)
print("\nTranslation Vectors:\n", tvecs)

Camera Calibrated:  0.4037687463614211

Camera Matrix:
 [[261.41353278   0.         331.06395374]
 [  0.         261.57312378 275.13052506]
 [  0.           0.           1.        ]]

Distortion Parameters:
 [[-0.04653193  0.7947218  -0.01212357  0.01361555 -1.54193315]]

Rotation Vectors:
 (array([[0.02893609],
       [0.2362934 ],
       [3.1120754 ]]), array([[-0.57373892],
       [ 0.09899432],
       [ 0.57675573]]))

Translation Vectors:
 (array([[-3.70993626],
       [ 2.35064719],
       [16.2358934 ]]), array([[-0.46435999],
       [-2.62885631],
       [16.37156468]]))


### Undistortion

Here I don't understand why undistortioned image looks like "fish-eye" effect.

In [5]:
#take new image
img = cv2.imread('chess/chess7.jpg')
h,  w = img.shape[:2]
newCameraMatrix, roi = cv2.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))

In [6]:
# undistort
dst = cv2.undistort(img, cameraMatrix, dist, None, newCameraMatrix)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv2.imwrite('calibrated/calibresult.png', dst)

True

### Re-Projection Error

In [7]:
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.05370872448116302


### 3D rendering using calibrated camera

Function which takes the corners in the chessboard (obtained using cv.findChessboardCorners()) and axis points to draw a 3D axis.

In [8]:
def draw(img, corners, imgpts):
    #convert coordinates to int
    corners = np.int32(corners)
    #take first corner
    corner = tuple(corners[0].ravel())
    imgpts = np.int32(imgpts)

    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

In [9]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:6,0:7].T.reshape(-1,2)
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)

In [11]:
for fname in glob.glob(os.path.join(chess_path, '*.jpg')):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (6,7),None)
    if ret == True:
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        # Find the rotation and translation vectors.
        ret, rvecs, tvecs = cv2.solvePnP(objp, corners2, cameraMatrix, dist)
        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, cameraMatrix, dist)
        # draw axes from the first found cormer to projected points in 3D
        img = draw(img, corners2, imgpts)
        cv2.imshow('img',img)
        k = cv2.waitKey(0)
        fname = fname.split("\\")[1].split(".")[0]
        cv2.imwrite("calibrated/"+fname+'_3d.png', img)
cv2.destroyAllWindows()